# 计算机视觉纳米学位项目

## 实战项目：图像标注

---

在该 notebook 中，你要学习的是如何从 [COCO 数据集](http://cocodataset.org/#home) 中对数据进行加载和预处理。此外，你还要设计一个CNN-RNN模型，使其自动生成图像标注。

请注意， **你对此notebook所做的任何修改，我们将不会对其进行评分**。  但是，你需要根据 **Step 3** 和 **Step 4** 中的说明，通过修改此项目的一部分，即**models.py**文件，从而实现你自己的CNN编码器和RNN解码器。我们将对你的**models.py** 文件进行评分。

点击以下链接，即可进入此 notebook：
- [Step 1](#step1): 了解数据加载器
- [Step 2](#step2): 使用数据加载器获取批次
- [Step 3](#step3): 使用CNN编码器进行实验
- [Step 4](#step4): 实现RNN解码器

<a id='step1'></a>
## Step 1: 了解数据加载器

我们已经编写了一个 [ 数据加载器](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) ，你可以使用它来批量加载COCO数据集。

在下面的代码单元格中，你可以使用 **data_loader.py** 中的`get_loader` 函数对数据加载器初始化。

> 在这个项目中，请不要修改 **data_loader.py** 文件，务必保留其原样。

 `get_loader` 函数将 **data_loader.py** 中可以用来探索的许多参数作为输入。现在，花一些时间在新窗口中打开 **data_loader.py**，并研究这些参数。大多数参数必须保留其默认值，只有以下参数的值允许被修改：
1. **`transform`** -  [图像转换 ](http://pytorch.org/docs/master/torchvision/transforms.html) 具体规定了应该如何对图像进行预处理，并将它们转换为PyTorch张量，然后再将它们用作CNN编码器的输入。在这里，我们建议你保留`transform_train`中提供的转换方法。之后，你可以选择自己的图像转换方法，对COCO图像进行预处理。
2. **`mode`** - `'train'`（用于批量加载训练数据）或 `'test'`（用于测试数据），二者中的一个。我们将分别说明数据加载器处于训练模式或测试模式的情况。参照该 notebook 中的说明进行操作时，请设置`mode='train'`.`'train'`，这样可以使数据加载器处于训练模式。
3. **`batch_size`** - 它是用于确定批次的大小。训练你的模型时，它是指图像标注对的数量，用于在每个训练步骤中修改模型权重。
4. **`vocab_threshold`** - 它是指在将单词用作词汇表的一部分之前，单词必须出现在训练图像标注中的总次数。在训练图像标注中出现少于`vocab_threshold` 的单词将被认为是未知单词。
5. **`vocab_from_file`** -  它是指一个布尔运算（Boolean），用于决定是否从文件中加载词汇表。

接下来，我们将更详细地描述`vocab_threshold` 和 `vocab_from_file`参数。现在，运行下面的代码单元格。要有耐心哦，可能需要几分钟才能运行！

In [1]:
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
!pip install nltk
import nltk
nltk.download('punkt')
from data_loader import get_loader
from torchvision import transforms

# Define a transform to pre-process the training images.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Set the minimum word count threshold.
vocab_threshold = 5

# Specify the batch size.
batch_size = 10

# Obtain the data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
loading annotations into memory...
Done (t=1.10s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=1.00s)
creating index...


  0%|          | 837/414113 [00:00<01:36, 4264.39it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:36<00:00, 4272.71it/s]


运行上面的代码单元格时，数据加载器会存储在变量`data_loader`中。

你可以将相应的数据集以`data_loader.dataset` 的方式访问。 此数据集是**data_loader.py**中`CoCoDataset`类的一个实例。 如果你对数据加载器和数据集感到陌生，我们建议你查看 [ 此 PyTorch 教程 ](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html)。

### 了解 `__getitem__` 方法

 `CoCoDataset`类中的`__getitem__`方法用于确定图像标注对在合并到批处理之前应如何进行预处理。 PyTorch中的所有`Dataset` 类都是如此。如果你此感到陌生，请查看 [上面链接中的教程 ](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html)。

当数据加载器处于训练模式时，该方法将首先获得训练图像的文件名（`path`）及其对应的标注（`caption`）。

#### 图像预处理 

图像预处理相对比较简单（来自`CoCoDataset`类中的`__getitem__`方法）：

In [3]:
import os
cocoapi_loc='/opt'
coco = COCO(os.path.join(cocoapi_loc,'cocoapi/annotations/captions_train2014.json'))

loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


In [5]:
coco

In [4]:
import nltk
ids = list(coco.anns.keys())
print(ids[0:5])
all_tokens = [nltk.tokenize.word_tokenize(str(coco.anns[ids[index]]['caption']).lower()) for index in [1,2]]
print(all_tokens)
#[len(token) for token in all_tokens]

[48, 67, 126, 148, 173]
[['a', 'panoramic', 'view', 'of', 'a', 'kitchen', 'and', 'all', 'of', 'its', 'appliances', '.'], ['a', 'blue', 'and', 'white', 'bathroom', 'with', 'butterfly', 'themed', 'wall', 'tiles', '.']]


In [5]:
ids = list(coco.anns.keys())
print(ids[0:5])
ann_id = ids[1]
print(ann_id)
img_id = coco.anns[ann_id]['image_id']
print(img_id)
path = coco.loadImgs(img_id)[0]['file_name']
print(path)

[48, 67, 126, 148, 173]
67
116100
COCO_train2014_000000116100.jpg


In [ ]:
# Convert image to tensor and pre-process using transform
img_folder = os.path.join(cocoapi_loc, 'cocoapi/images/train2014/')
image = Image.open(os.path.join(img_folder, path)).convert('RGB')
image = self.transform(image)

将训练文件夹`path`中的图像进行加载后，你需要使用与在实例化数据加载器时相同的转换方法（`transform_train`）对这些图像进行预处理。

#### 标注预处理 

图像的标注也需要进行预处理，并为训练做好准备。 在这个例子中，为了生成图像标注，我们的目标是创建一个模型，该模型是用于根据一个句子的前一个token预测下一个token。因此，我们要把与所有图像相关联的标注转换为标记化单词列表，然后将其转换为可用于训练网络的PyTorch张量。

为了更详细地了解COCO描述是如何进行预处理的，我们首先需要看一下`CoCoDataset`类的`vocab`实例变量。下面的代码片段是从 `CoCoDataset`类中的`__init__`方法中提取的：

In [ ]:
def __init__(self, transform, mode, batch_size, vocab_threshold, vocab_file, start_word, 
        end_word, unk_word, annotations_file, vocab_from_file, img_folder):
        ...
        self.vocab = Vocabulary(vocab_threshold, vocab_file, start_word,
            end_word, unk_word, annotations_file, vocab_from_file)
        ...

从上面的代码片段中，你可以看到，`data_loader.dataset.vocab`是**vocabulary.py**中`Vocabulary` 类的一个实例。 现在，花几分钟的时间查看一下**data_loader.py**中的完整代码，然后自行验证一下吧。

接下来，我们要使用这个实例对COCO描述进行预处理（来自`CoCoDataset`类中的`__getitem__`方法）：

In [ ]:
# Convert caption to tensor of word ids.
tokens = nltk.tokenize.word_tokenize(str(caption).lower())   # line 1
caption = []                                                 # line 2
caption.append(self.vocab(self.vocab.start_word))            # line 3
caption.extend([self.vocab(token) for token in tokens])      # line 4
caption.append(self.vocab(self.vocab.end_word))              # line 5
caption = torch.Tensor(caption).long()                       # line 6

你很快就会看到，此代码会将所有字符串值的标注转换为整数列表，然后再将其转换为PyTorch张量。 为了弄清楚此代码的工作原理，我们将其应用于下一个代码单元格中的示例标注。

In [6]:
sample_caption = 'A person doing a trick on a rail while riding a skateboard.'

在代码片段的**`line 1`**中，标注中的每个字母都转换为小写，且[`nltk.tokenize.word_tokenize`](http://www.nltk.org/) 函数用于获取字符串值token的列表。 运行下一个代码单元格，将其对`sample_caption`的影响可视化。

In [7]:
import nltk

sample_tokens = nltk.tokenize.word_tokenize(str(sample_caption).lower())
print(sample_tokens)

['a', 'person', 'doing', 'a', 'trick', 'on', 'a', 'rail', 'while', 'riding', 'a', 'skateboard', '.']


在**`line 2`**和**`line 3`**中，我们初始化一个空列表并附加一个整数来标记一个图像标注的开头。 我们建议你阅读的 [这篇论文](https://arxiv.org/pdf/1411.4555.pdf) 使用了一个特殊的起始单词（与一个特殊的结束单词，我们将在下面查看）来标记一个标注的开头（和结尾）。

这个特殊的起始单词（`"<start>"`）是在实例化数据加载器时确定的，并作为参数（`start_word`）传递。 你**需要**将此参数保持为其默认值（`start_word="<start>"`）。

你将在下面看到，整数`0`始终用于标记一个标注的开头。

In [16]:
sample_caption = []

start_word = data_loader.dataset.vocab.start_word
print('Special start word:', start_word)
sample_caption.append(data_loader.dataset.vocab(start_word))
print(sample_caption)

Special start word: <start>
[0]


在**`line 4`**中，我们通过添加与标注中的每个token对应的整数来继续这个列表。

In [17]:
sample_caption.extend([data_loader.dataset.vocab(token) for token in sample_tokens])
print(sample_caption)

[0, 3, 98, 754, 3, 396, 39, 3, 1009, 207, 139, 3, 753, 18]


在**`line 5`**，我们附加了最后一个整数，用来标记该标注的结尾。

与上面提到的特殊起始单词相同，特殊结束单词（`"<end>"`）会在实例化数据加载器时被确定，并作为参数（`end_word`）传递。 你**需要**将此参数保持为其默认值（`end_word="<end>"`）。

你将在下面看到，整数`1`始终用于标记一个标注的结尾。

In [18]:
end_word = data_loader.dataset.vocab.end_word
print('Special end word:', end_word)

sample_caption.append(data_loader.dataset.vocab(end_word))
print(sample_caption)

Special end word: <end>
[0, 3, 98, 754, 3, 396, 39, 3, 1009, 207, 139, 3, 753, 18, 1]


最后，在**`line 6`**中，我们将整数列表转换为PyTorch张量并将其转换为 [long 类型](http://pytorch.org/docs/master/tensors.html#torch.Tensor.long)。 此外，你可以在 [这个网站](http://pytorch.org/docs/master/tensors.html)上阅读有关不同类型PyTorch张量的更多信息。

In [19]:
import torch
#sample_caption = torch.Tensor(sample_caption)
sample_caption = torch.Tensor(sample_caption).long()
#print(sample_caption)
print(sample_caption.shape)

torch.Size([15])


就是这样！ 总之，所有标注都会转换为token列表，其中， _特殊的_开始和结束token用来标记句子的开头和结尾，如下所示：

In [ ]:
[<start>, 'a', 'person', 'doing', 'a', 'trick', 'while', 'riding', 'a', 'skateboard', '.', <end>]

然后将此token列表转换为整数列表，其中，词汇表中的每个不同单词都具有各自相关联的整数值：

In [12]:
[0, 3, 98, 754, 3, 396, 207, 139, 3, 753, 18, 1]

[0, 3, 98, 754, 3, 396, 207, 139, 3, 753, 18, 1]

最后，此列表将转换为一个PyTorch张量。 使用上述**`lines 1-6`**的相同步骤对COCO数据集中的所有标注进行预处理。

如你所看到的那样，为了将token转换为其对应的整数，我们将`data_loader.dataset.vocab` 称作一个函数。 你可以在**vocabulary.py**中`Vocabulary`类的`__call__`方法中详细了解此call具体是如何工作的。

In [ ]:
def __call__(self, word):
    if not word in self.word2idx:
        return self.word2idx[self.unk_word]
    return self.word2idx[word]

`word2idx`实例变量是一个Python [字典](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) ，它由字符串值键索引，而这些字符串值键主要是从训练标注获得的token。 对于每个键，对应的值是token在预处理步骤中映射到的整数。

使用下面的代码单元格查看该字典的子集。

In [20]:
# Preview the word2idx dictionary.
dict(list(data_loader.dataset.vocab.word2idx.items())[:10])

{'<start>': 0,
 '<end>': 1,
 '<unk>': 2,
 'a': 3,
 'very': 4,
 'clean': 5,
 'and': 6,
 'well': 7,
 'decorated': 8,
 'empty': 9}

此外，我们还输出了键总数。

In [2]:
# Print the total number of keys in the word2idx dictionary.
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 8855


接下来，你会看到，如果检查**vocabulary.py**中的代码，则通过遍历训练数据集中的图像标注就可以创建一个`word2idx`字典。 如果token在训练集中出现的次数不小于`vocab_threshold`次数，则将其作为键添加到该字典中并分配一个相应的唯一整数。 之后，你可以选择在实例化数据加载器时修改`vocab_threshold`参数。 请注意，通常情况下，**较小的**`vocab_threshold`值会在词汇表中生成**更多的**token。 另外，我们建议你在创建新数据加载器之前减少`vocab_threshold`的值，这样便于在下一个代码单元格中自行检查。

In [3]:
# Modify the minimum word count threshold.
vocab_threshold = 3

# Obtain the data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

loading annotations into memory...
Done (t=0.93s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...


  0%|          | 811/414113 [00:00<01:44, 3946.24it/s]

Done (t=0.94s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:37<00:00, 4265.94it/s]


In [4]:
# Print the total number of keys in the word2idx dictionary.
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 11498


`word2idx` 字典中还有一些特殊键。 通过前面的内容，你已经熟悉了特殊的起始单词（`"<start>"`）和特殊的结束单词（`"<end>"`）。在这里，还有一个特殊的token，对应的是未知的单词（`"<unk>"`）。 所有未出现在`word2idx`字典中的token都被视为未知单词。 在预处理步骤中，任何未知token都会映射到整数`2`。

In [16]:
unk_word = data_loader.dataset.vocab.unk_word
print('Special unknown word:', unk_word)

print('All unknown words are mapped to this integer:', data_loader.dataset.vocab(unk_word))

Special unknown word: <unk>
All unknown words are mapped to this integer: 2


请在下面通过对从未出现在训练标注中的提供的无意义单词进行预处理，做个自行检查吧。

In [17]:
print(data_loader.dataset.vocab('jfkafejw'))
print(data_loader.dataset.vocab('ieowoqjf'))

2
2


最后提到的是创建数据加载器时提供的`vocab_from_file`参数。请注意，要理解这个参数，在创建新的数据加载器时，词汇表（`data_loader.dataset.vocab`）需要保存为项目文件夹中的 [pickle](https://docs.python.org/3/library/pickle.html)文件，文件名为`vocab.pkl`。

如果你此刻还在调整`vocab_threshold`参数的值，则**必须**设置为`vocab_from_file=False`，这样才能使更改生效。

但是，如果你对为`vocab_threshold`参数选定的值感到满意，则只需*再次*使用所选的`vocab_threshold`运行数据加载器即可，这样可以将新词汇表保存到文件中。然后，就可以设置`vocab_from_file=True` 了，这样便于在文件中加载词汇表并加速数据加载器的实例化。请注意，从零开始构建词汇表是实例化数据加载器过程中最耗时的一部分，因此我们强烈建议你尽快设置`vocab_from_file=True`。

另外，还需要注意的是，如果`vocab_from_file=True`，则在实例化数据加载器时为`vocab_threshold`提供的任何参数都将被完全忽略。

In [24]:
# Obtain the data loader (from file). Note that it runs much faster than before!
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_from_file=True)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 776/414113 [00:00<01:43, 3988.81it/s]

Done (t=0.89s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:36<00:00, 4281.67it/s]


在下一部分中，你将要学习的是如何使用数据加载器来获取批量训练数据。

<a id='step2'></a>
## Step 2: 使用数据加载器获取批量数据

数据集中的图像标注长度差异很大，查看一下Python列表`data_loader.dataset.caption_lengths`就可以发现这一点。在这个列表中，每个训练标注都有一个entry（其中，值用于存储相应标注的长度）。

在下面的代码单元格中，我们使用此列表输出每个长度的训练数据中的标注总数。 接下来你会看到，大多数标注的长度为10。同时，过短与过长的标注非常少见。

In [25]:
import nltk
from collections import Counter

# Tally the total number of training captions with each length.
counter = Counter(data_loader.dataset.caption_lengths)
lengths = sorted(counter.items(), key=lambda pair: pair[1], reverse=True)
for value, count in lengths:
    print('value: %2d --- count: %5d' % (value, count))

value: 10 --- count: 86334
value: 11 --- count: 79948
value:  9 --- count: 71934
value: 12 --- count: 57637
value: 13 --- count: 37645
value: 14 --- count: 22335
value:  8 --- count: 20771
value: 15 --- count: 12841
value: 16 --- count:  7729
value: 17 --- count:  4842
value: 18 --- count:  3104
value: 19 --- count:  2014
value:  7 --- count:  1597
value: 20 --- count:  1451
value: 21 --- count:   999
value: 22 --- count:   683
value: 23 --- count:   534
value: 24 --- count:   383
value: 25 --- count:   277
value: 26 --- count:   215
value: 27 --- count:   159
value: 28 --- count:   115
value: 29 --- count:    86
value: 30 --- count:    58
value: 31 --- count:    49
value: 32 --- count:    44
value: 34 --- count:    39
value: 37 --- count:    32
value: 33 --- count:    31
value: 35 --- count:    31
value: 36 --- count:    26
value: 38 --- count:    18
value: 39 --- count:    18
value: 43 --- count:    16
value: 44 --- count:    16
value: 48 --- count:    12
value: 45 --- count:    11
v

为了生成批量的训练数据，我们首先对标注长度进行采样。在采样中，抽取的所有长度的概率需要与数据集中具有该长度的标注的数量成比例。 然后，我们检索一批图像标注对的size`batch_size`，其中，所有标注都具有采样长度。 这种用于分配批次的方法与 [这篇文章 ](https://arxiv.org/pdf/1502.03044.pdf) 中的过程相匹配，并且已被证明在不降低性能的情况下具有计算上的有效性。

运行下面的代码单元格，生成一个批次。 `CoCoDataset`类中的`get_train_indices`方法首先对标注长度进行采样，然后对与训练数据点对应的`batch_size`indices进行采样，并使用该长度的标注。 这些indices存储在`indices`下方。

这些indices会提供给数据加载器，然后用于检索相应的数据点。该批次中的预处理图像和标注存储在`images`和`captions`中。

In [26]:
import numpy as np
caption_lengths = data_loader.dataset.caption_lengths
sel_length = np.random.choice(caption_lengths)
caption_lengths
caption_lengths[2] 
np.where([caption_lengths[i] == sel_length for i in np.arange(len(caption_lengths))])
np.where([caption_lengths[i] == sel_length for i in np.arange(len(caption_lengths))])[0]
#print(x.shape)

array([     3,      7,      8, ..., 414099, 414110, 414112])

In [27]:
import numpy as np
import torch.utils.data as data

# Randomly sample a caption length, and sample indices with that length.
indices = data_loader.dataset.get_train_indices()
print('sampled indices:', indices)

# Create and assign a batch sampler to retrieve a batch with the sampled indices.
new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler
    
# Obtain the batch.
images, captions = next(iter(data_loader))
    
print('images.shape:', images.shape)
print('captions.shape:', captions.shape)

# (Optional) Uncomment the lines of code below to print the pre-processed images and captions.
#print('images:', images)
#print('captions:', captions)

sampled indices: [273457, 48026, 368829, 156477, 390499, 406564, 88367, 123249, 289586, 288417]
images.shape: torch.Size([10, 3, 224, 224])
captions.shape: torch.Size([10, 12])


In [28]:
import torch.nn as nn
vocab_size = len(data_loader.dataset.vocab)
embed_size = 256
embedding_word = nn.Embedding(vocab_size, embed_size)
print(embedding_word)
print(captions.shape)
#print(captions[:,: 12].shape)
#embeds_word = embedding_word(captions[: , : 1])
#features.unsqueeze(1).shape


Embedding(11498, 256)
torch.Size([10, 12])


每次运行上面的代码单元格时，都会对不同的标注长度进行采样，并返回不同批次的训练数据。多次运行代码单元格，尝试检验一下吧！

在接下来的一个notebook（**2_Training.ipynb**）中训练你的模型。我们会将用于生成训练批次的代码提供给你。

> 打开下面的一个notebook（**2_Training.ipynb**）之前，我们强烈建议你花些时间熟悉**data_loader.py**和**vocabulary.py**中的代码。本notebook的**Step 1**和**Step 2**主要做了一些基本的介绍并指导你的理解。但是，我们的说明并非详尽无遗，作为项目的一部分，是否要学习如何最好地利用这些文件来完成项目，这就取决于你了。__但你不可以修改 *data_loader.py* 或 *vocabulary.py*中的任何代码哦。__

在接下来的步骤中，我们将重点学习如何在PyTorch中指定一个CNN-RNN架构，从而实现最终的图像标注目标。

<a id='step3'></a>
## Step 3: 使用CNN编码器进行实验

运行下面的代码单元格，从**model.py**中导入`EncoderCNN`和`DecoderRNN`。

In [30]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

# Import EncoderCNN and DecoderRNN. 
from model import EncoderCNN, DecoderRNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


在下一个代码单元格中，我们定义了一个`device`，你将使用它将PyTorch张量移动到GPU（如果CUDA可用的话）。 在进行下一步之前，运行此代码单元格。

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

运行下面的代码单元格，在`encoder`中实例化CNN编码器。

然后，该notebook的 **Step 2**中批次的预处理图像会通过编码器，且其输出会存储在`features`中。

In [32]:
# Specify the dimensionality of the image embedding.
embed_size = 256

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Initialize the encoder. (Optional: Add additional arguments if necessary.)
encoder = EncoderCNN(embed_size)

# Move the encoder to GPU if CUDA is available.
encoder.to(device)
    
# Move last batch of images (from Step 2) to GPU if CUDA is available.   
images = images.to(device)

# Pass the images through the encoder.
features = encoder(images)

print('type(features):', type(features))
print('features.shape:', features.shape)

# Check that your encoder satisfies some requirements of the project! :D
assert type(features)==torch.Tensor, "Encoder output needs to be a PyTorch Tensor." 
assert (features.shape[0]==batch_size) & (features.shape[1]==embed_size), "The shape of the encoder output is incorrect."

type(features): <class 'torch.Tensor'>
features.shape: torch.Size([10, 256])


我们提供给你的编码器使用预先训练的ResNet-50架构（删除了最终的完全连接层）从一批预处理图像中提取特征。然后将输出展平为矢量，然后通过 `Linear`层，将特征向量转换为与单词向量同样大小的向量。

![Encoder](images/encoder.png)

为了试验其他架构，我们欢迎并鼓励你修改 **model.py**中的编码器。我们特别建议你 [ 使用一个不同的预训练模型架构 ](http://pytorch.org/docs/master/torchvision/models.html)。当然了，你也可以考虑[ 添加批次归一化 ](http://pytorch.org/docs/master/nn.html#normalization-layers)。

> 但是，你**无**需更改编码器的任何内容。

对于此项目，你**必须**将预先训练好的CNN合并到编码器中。`EncoderCNN`类必须将`embed_size`作为一个输入参数，这个参数也将对应于你将在 Step 4 中实现的RNN解码器输入的维度。在接下来的notebook中训练模型时（**2_Training.ipynb**） ，我们鼓励你对`embed_size`的值进行调整哦。

如果你决定修改`EncoderCNN`类，请保存**model.py**并重新执行上面的代码单元格。如果该代码单元格返回一个断言错误，请在进行下一步之前按照说明修改代码。这个断言错误可以确保`features`具有一个形状为`[batch_size, embed_size]`的PyTorch张量。

<a id='step4'></a>
## Step 4: 实现RNN解码器

在执行下一个代码单元格之前，必须在**model.py**中的`DecoderRNN` 类中编写`__init__`和 `forward`方法。 （ **不要**编写`sample`方法，但到 notebook **3_Inference.ipynb**时，可以使用此方法。）

>  `DecoderRNN`类中`__init__`的和 `forward`方法是你需要在此notebook中修改的唯一内容。你将在接下来出现的notebook中编写更多实现方式。

你的解码器将会是`DecoderRNN`类的一个实例，且必须接收下列输入：
- 包含嵌入图像特征的PyTorch张量`features`（在 Step 3 中输出，当 Step 2 中的最后一批图像通过编码器时）
- 与 Step 2中最后一批标注（`captions`）相对应的PyTorch张量。

请注意，我们编写数据加载器的方式应该会简化你的代码。特别是，每个训练批次都包含预处理的标注，其中所有标注都具有相同的长度（`captions.shape[1]`），因此**你无需担心填充问题**。
> 虽然我们鼓励你实现 [本文](https://arxiv.org/pdf/1411.4555.pdf)中描述的解码器，但仍然希望你实现自行选择的任何一种架构，只要至少使用一个RNN层，且隐藏维度为`hidden_size`。

虽然你将使用当前存储在notebook中的最后一个批次来测试该解码器，但你的解码器应编写为接收嵌入图像特征和预处理标注的任意批次作为输入，其中所有标注具有相同的长度。

![Decoder](images/decoder.png)

 在下面的代码单元格中，`outputs`应该是一个大小为`[batch_size, captions.shape[1], vocab_size]`的PyTorch张量。这样设计输出的目的是`outputs[i,j,k]`包含模型的预测分数，而该分数表示批次中第 `i`个标注中的第`j`个token是词汇表中第`k`个token的可能性。在接下来的notebook（**2_Training.ipynb**）中，我们会提供代码，将这些分数提供给PyTorch中的 [`torch.nn.CrossEntropyLoss`](http://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss) 优化程序。

In [33]:
import torch.nn as nn
vocab_size = len(data_loader.dataset.vocab)
embed_size = 256
embedding_word = nn.Embedding(vocab_size, embed_size)
print(embedding_word)
captions = captions.to(device)
print(captions.shape)
print(captions[:,: -1].shape)
#embeds_word = embedding_word(captions)
features.unsqueeze(1).shape

Embedding(11498, 256)
torch.Size([10, 12])
torch.Size([10, 11])


torch.Size([10, 1, 256])

In [34]:
# Specify the number of features in the hidden state of the RNN decoder.
hidden_size = 512

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Store the size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the decoder.
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move the decoder to GPU if CUDA is available.
decoder.to(device)
    
# Move last batch of captions (from Step 1) to GPU if CUDA is available 
captions = captions.to(device)

# Pass the encoder output and captions through the decoder.
outputs = decoder(features, captions)

print('type(outputs):', type(outputs))
print('outputs.shape:', outputs.shape)

# Check that your decoder satisfies some requirements of the project! :D
assert type(outputs)==torch.Tensor, "Decoder output needs to be a PyTorch Tensor."
assert (outputs.shape[0]==batch_size) & (outputs.shape[1]==captions.shape[1]) & (outputs.shape[2]==vocab_size), "The shape of the decoder output is incorrect."

type(outputs): <class 'torch.Tensor'>
outputs.shape: torch.Size([10, 12, 11498])


在接下来的notebook中训练模型时（**2_Training.ipynb**） ，我们鼓励你对`embed_size`的值进行调整哦。